In [ ]:
import sys
import os
import warnings
import pandas as pd
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

In [ ]:
import pytest
import pyarrow.parquet as pq
import pyarrow.fs as fs
import pyarrow.dataset as ds
import s3fs as fs3
import pyarrow as pa

In [ ]:
import daphmeIO
import constants

# Test formats and folder structures

## Generate test dataset

## Import partitioned dataset (data 3)

In [ ]:
column_names = pd.read_csv('../../data/sample2/sample2.csv', nrows=0).columns
column_names

In [ ]:
dataset = ds.dataset(path, format="parquet", partitioning="hive")
df = dataset.to_table().to_pandas()
df 

## Functions that check integrity

kind of based on scikit-mobility

In [ ]:
import pyarrow.parquet as pq
import pandas as pd
from functools import partial
import multiprocessing
from multiprocessing import Pool
import re
from pyspark.sql import SparkSession
# from . import constants
import constants

def get_pq_users(path: str, id_string: str):
    return pq.read_table(path, columns=[id_string]).column(id_string).unique().to_pandas()

def get_pq_user_data(path: str, users: list[str], id_string: str):
    return pq.read_table(path, filters=[(id_string, 'in', users)]).to_pandas()

def _read_partitioned_pq(path):
    return None

def _update_schema(original, new_labels):
    updated_schema = dict(original)
    for label in new_labels:
        if label in constants.SCHEMA_NAMES:
            updated_schema[label] = new_labels[label]
    return updated_schema

def _has_traj_cols(df, traj_cols):
    
    # Check for sufficient spatial columns
    spatial_exists = (
        ('latitude' in traj_cols and 'longitude' in traj_cols and 
         traj_cols['latitude'] in df and traj_cols['longitude'] in df) or
        ('x' in traj_cols and 'y' in traj_cols and 
         traj_cols['x'] in df and traj_cols['y'] in df) or
        ('geohash' in traj_cols and traj_cols['geohash'] in df)
    )
    
    # Check for sufficient temporal columns
    temporal_exists = (
        ('datetime' in traj_cols and traj_cols['datetime'] in df) or
        ('timestamp' in traj_cols and traj_cols['timestamp'] in df)
    )
    
    if not spatial_exists:
        raise ValueError(
            "Could not find required spatial columns in {}. The dataframe columns must contain or map to at least one of the following sets: "
            "('latitude', 'longitude'), ('x', 'y'), or 'geohash'.".format(df.columns.tolist())
        )
        
    if not temporal_exists:
        raise ValueError(
            "Could not find required temporal columns in {}. The dataframe columns must contain or map to either 'datetime' or 'timestamp'.".format(df.columns.tolist())
        )
    
    return spatial_exists and temporal_exists


def _cast_traj_cols(df, traj_cols):
    if 'datetime' in traj_cols and traj_cols['datetime'] in df:
        if not pd.core.dtypes.common.is_datetime64_any_dtype(df[traj_cols['datetime']].dtype):
            df[traj_cols['datetime']] = pd.to_datetime(df[traj_cols['datetime']])
    if 'timestamp' in traj_cols and traj_cols['timestamp'] in df:
        # Coerce to integer if it's not already
        if not pd.core.dtypes.common.is_integer_dtype(df[traj_cols['timestamp']].dtype):
            df[traj_cols['timestamp']] = df[traj_cols['timestamp']].astype(int)

    float_cols = ['latitude', 'longitude', 'x', 'y']
    for col in float_cols:
        if col in traj_cols and traj_cols[col] in df:
            if not pd.core.dtypes.common.is_float_dtype(df[traj_cols[col]].dtype):
                df[traj_cols[col]] = df[traj_cols[col]].astype("float")

    string_cols = ['user_id', 'geohash']
    for col in string_cols:
        if col in traj_cols and traj_cols[col] in df:
            if not pd.core.dtypes.common.is_string_dtype(df[traj_cols[col]].dtype):
                df[traj_cols[col]] = df[traj_cols[col]].astype("str")

    return df

def _is_traj_df(df, traj_cols = None, **kwargs):
    
    if not (isinstance(df, pd.DataFrame) or isinstance(df, gpd.GeoDataFrame)):
        return False
    
    if not traj_cols:
        traj_cols = {}
        traj_cols = _update_schema(traj_cols, kwargs) #kwargs ignored if traj_cols
        
    traj_cols = _update_schema(constants.DEFAULT_SCHEMA, traj_cols)
    
    if not _has_traj_cols(df, traj_cols):
        return False
    
    if 'datetime' in traj_cols and traj_cols['datetime'] in df:
        if not pd.core.dtypes.common.is_datetime64_any_dtype(df[traj_cols['datetime']].dtype):
            return False
    elif 'timestamp' in traj_cols and traj_cols['timestamp'] in df:
        if not pd.core.dtypes.common.is_integer_dtype(df[traj_cols['timestamp']].dtype):
            return False

    float_cols = ['latitude', 'longitude', 'x', 'y']
    for col in float_cols:
        if col in traj_cols and traj_cols[col] in df:
            if not pd.core.dtypes.common.is_float_dtype(df[traj_cols[col]].dtype):
                return False

    string_cols = ['user_id', 'geohash']
    for col in string_cols:
        if col in traj_cols and traj_cols[col] in df:
            if not pd.core.dtypes.common.is_string_dtype(df[traj_cols[col]].dtype):
                return False

    return True


def from_pandas(df, traj_cols = None, spark_enabled=False, **kwargs):

    if not (isinstance(df, pd.DataFrame) or isinstance(df, gpd.GeoDataFrame)):
        raise TypeError(
            "Expected the data argument to be either a pandas DataFrame or a GeoPandas GeoDataFrame."
        )
    
    # valid trajectory column names passed to **kwargs collected
    if not traj_cols:
        traj_cols = {}
        traj_cols = _update_schema(traj_cols, kwargs) #kwargs ignored if traj_cols
            
    for key, value in traj_cols.items():
        if value not in df:
            warnings.warn(f"Trajectory column '{value}' specified for '{key}' not found in df.")
            
    # include defaults when missing
    traj_cols = _update_schema(constants.DEFAULT_SCHEMA, traj_cols)
    
    if _has_traj_cols(df, traj_cols):
        return _cast_traj_cols(df, traj_cols)

def from_file(filepath, format='csv', traj_cols = None, **kwargs):
    assert format in ['parquet', 'csv']

    if format=='parquet':
        dataset = ds.dataset(filepath, format="parquet", partitioning="hive", **kwargs)
        df = dataset.to_table().to_pandas()
        return from_pandas(df, traj_cols = None, **kwargs)
        
    elif format=='csv':
        if os.path.isdir(filepath):
            dataset = ds.dataset(filepath, format="csv", partitioning="hive", **kwargs)
            df = dataset.to_table().to_pandas()
            return from_pandas(df, traj_cols = None, **kwargs)
    
        else:
            # Pass only valid kwargs for pandas.read_csv
            read_csv_args = inspect.signature(pd.read_csv).parameters
            read_csv_kwargs = {k: v for k, v in kwargs.items() if k in read_csv_args}
            
            df = pd.read_csv(filepath, **read_csv_kwargs)
            return from_pandas(df, traj_cols = traj_cols, **kwargs)
    
    return None

In [ ]:
data = pd.DataFrame([[1, 39.984094, 116.319236, '2008-10-23 13:53:05'],
 [1, 39.984198, 116.319322, '2008-10-23 13:53:06'],
 [1, 39.984224, 116.319402, '2008-10-23 13:53:11'],
 [1, 39.984211, 116.319389, '2008-10-23 13:53:16']], columns = ['uid', 'latitude', 'longitude', 'time'])

In [ ]:
traj_cols = {'user_id':'uid',
         'latitude':'latitude',
         'longitude':'longitude',
            'datetime':'time'}
df = from_object(data, traj_cols)

In [ ]:
import pyarrow as pa
import pandas as pd

# Creating a sample pyarrow table
data = {
    'user_id': [1, 2, 2, 3, 3, 3],
    'value': [10, 20, 30, 40, 50, 60]
}

# Creating a pandas DataFrame
df = pd.DataFrame(data)

# Converting pandas DataFrame to pyarrow Table
table = pa.Table.from_pandas(df, preserve_index=False)

# Extracting the user_id column and converting it to pandas Series
user_id_col = table['user_id', 'value']
user_ids = user_id_col.to_pandas()

# Displaying the type and first few entries of user_ids
print(type(user_ids))  # Should return <class 'pandas.core.series.Series'>
print(user_ids.head())  # Displays the first few entries